# Delta

A declarative description of Operational Transform (OT) operations developed with the context of Quill's Delta format for representation of rich text content changes.

For details, please see:

- https://quilljs.com/guides/designing-the-delta-format/
- https://quilljs.com/docs/delta/

We get started with our cluster of imports.

In [5]:
import dataclasses
from collections import deque
from weakref import proxy
from typing import Any, ClassVar, Dict, Optional

## Operation

Now we will define the basic model for our "operation", in the abstract.

In [28]:
@dataclasses.dataclass
class Operation:
	OPS = {}
	
	op:ClassVar[str]
	embed:str
	value:Any
	attributes:Optional[Dict[str,str]]
	
	def __init__(self, *args, **kw):
		"""Initialize an operation using a shortcut syntax."""
		attributes = kw.get('attributes', {})
		attributes.update({k: v for k, v in kw.items() if k not in ('op', 'embed', 'value', 'attributes')})
		kw['attributes'] = attributes
		super().__init__()
	
	@classmethod
	def register(cls, target):
		"""Register a new operation type.
		
		We could use a metaclass for this, and Marrow Schema exposes an __attributed__ callback.
		"""
		cls.OPS.setdefault(target.op, proxy(target))
		return target
	
	@classmethod
	def from_json(cls, value):
		"""For ingress interoperability with the Delta format."""
		
		value = dict(value)
		
		instance = cls()
		instance.op, = set(value) - {'attributes'}
		instance.value = value.pop(instance.op)
		instance.attributes = value.get('attributes', {})
		
		# if not isinstance(instance, self.OPS[instance.op]):
		# 	self.OPS[instance.op].from_json(value)
		
		return instance
	
	@property
	def as_json(self):
		"""For egress interoperability with the Delta format."""
		
		value = {self.embed: self.value} if self.embed else self.value
		result = {self.op: value}
		
		if self.attributes:
			result['attributes'] = self.attributes
		
		return result
	
	@property
	def length(self):  # Optimization for delete/retain cases.
		return self.value

And now we can get into the gritty details of implementations for the specific operations required.

## Insert

A representation of the addition of a block of content to the stream.

Define the text inserted as the first positional parameter, and optional attributes as keyword arguments.

In the resulting JSON, only insert operations have an `insert` key defined. A string value represents inserting text. Any other type represents inserting an embed, please reference the `Embed` class below. In both cases an optional `attributes` key can be defined with an embedded object to describe additonal formatting information. Formats can be changed by the retain operation, see `Retain` below.
	
	Insert("Gandalf", bold=True).as_json
	{"insert": "Gandalf", "attributes": {"bold": true}}

In [45]:
@Operation.register
class Insert(Operation):
	op = 'insert'
	
	@property
	def length(self):
		"""Return the textual length of the content represented by this insert."""
		
		return len(self.value)
	
	@classmethod
	def from_json(cls, value):
		"""Transform a JSON-encoded operation into an Operation instance.
		
		This specific version can differentiate between `Insert` and `Embed` constructs.
		"""
		
		instance = super().from_json(value)
		
		if instance.embed or isinstance(instance.value, str):
			return instance
		
		return Embed.from_json(value)

In [46]:
Insert("Google", link="https://www.google.com/")  # Insert a link.

AttributeError: 'Insert' object has no attribute 'embed'

In [47]:
@Operation.register
class Embed(Insert):
	"""A richer description of inserted non-textual content.
	
		Embed('image', 'https://octodex.github.com/images/labtocat.png', alt="Lab Octocat")
	
	Embed('image', '/assets/img/icon.png', link='/').as_json
		{"insert": {"image": "/assets/img/icon.png"}, "attributes": {"link": "/"}}
	"""
	
	embed = dict
	length = 1  # The length of any embed is always one.

In [48]:
@Operation.register
class Delete(Operation):
	"""Identify characters to delete.
	
	Delete(10)  # Delete the next 10 characters.
	"""
	
	op = 'delete'
	value:int
	attributes = None  # Deletions have no additional attributes.

In [49]:
@Operation.register
class Retain(Operation):
	"""Preserve a range of characters, optionally with adjusted attributes.
	
	# Unbold and italicize "Gandalf".
	Retain(7, bold=None, italic=True).as_json
		{"retain": 7, "attributes": {"bold": null, "italic": true}}
	
	# Now keep " the ", insert "White", and delete "Grey".
	Retain(5)
	Insert("White", color="#fff")
	Delete(4)
	"""
	
	op = 'retain'
	value:int

In [50]:
class Delta:
	ops:list
	
	def __init__(self):
		self.ops = []
	
	def insert(self, value, **attributes):
		"""Append an insert operation while maintaining the ability to chain calls.
		
		Please refer to the docstring of the `Insert` class for details.
		"""
		
		self.ops.append(Insert(value, **attributes))
		return self
	
	def embed(self, kind, value, **attributes):
		"""Append an embed insert operation while maintaining the ability to chain calls.
		
		Please refer to the docstring of the `Embed` class for details.
		"""
		
		self.ops.append(Embed(kind, value, **attributes))
		return self
	
	def retain(self, count, **attributes):
		"""Append a retain operation while maintaining the ability to chain calls.
		
		Please refer to the docstring of the `Retain` class for details.
		"""
		
		self.ops.append(Retain(count, **attributes))
		return self
	
	def delete(self, count):
		"""Append a delete operation while maintaining the ability to chain calls.
		
		Please refer to the docstring of the `Delete` class for details.
		"""
		self.ops.append(Delete(count))
	
	def apply(self, delta):
		pass
	
	def diff(self, other):
		pass
	
	@property
	def fragments(self):
		index = 0
		
		for fragment in self.ops:
			yield index, fragment
			index += fragment.length
	
	@property
	def lines(self):
		"""A read-only view of an at-rest document.
		
		For efficiency sake this repeatedly fills and drains a buffer to gather elements for a line, then yields the
		paragraph attributes and an iterator across that buffer prior to clearing it.
		"""
		
		buf = []
		
		for offset, fragment in self.fragments:
			if fragment.op != 'insert':
				raise ValueError("Not a root document, encountered: " + fragment.op)
			
			if not isinstance(fragment.value, str) or fragment.value != b"\n":
				buf.append(fragment)
				continue
			
			yield fragment.attributes, iter(buf)
			buf.clear()

In [51]:
document = Delta().insert("Hello", bold=True).insert(" world!")
document.insert("\n", align='right')
document.insert("This is a demo of Delta storage.")
document.insert("\n", align='left')
document.embed('image', 'monkey.png', alt="Funny monkey picture.")

for attrs, fragments in document.lines:
	line = "".join(unciode(fragment.value) for offset, fragment in fragments)
	
	if 'align' in attrs:
		if attrs['align'] == 'right':
			line = line.rjust(80)
	
	print(line)

print()

# Construct a document wiht the text "Gandalf the Grey", Gandalf in bold, Grey in grey.
document = Delta().insert("Gandalf", bold=True).insert(" the ").insert("Grey", color='#ccc')

# Change the text and representative color.
death = Delta().retain(12).delete(4).insert("White", color='#fff')

document.apply(death)

AttributeError: 'Insert' object has no attribute 'value'